In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END


In [2]:
class TransactionRequest(TypedDict):
    action:str
    opening_balance:int
    amount:int
    new_balance:int

In [3]:
def categorize_transaction(request:TransactionRequest)->str:
    print(f"Transaction Request:{request}")
    if request['action'] == 'deposit':
        return 'deposit'
    elif request['action'] == 'withdraw':
        return 'withdraw'
    else:
        return 'unknown'

In [4]:
def handle_deposit(request: TransactionRequest)->TransactionRequest:
    new_bal = request['opening_balance']+request['amount']
    print(f"Depositing {request['amount']}-> New balance:{new_bal}")
    request['new_balance'] = new_bal
    return request

In [5]:
def handle_withdraw(request: TransactionRequest)->TransactionRequest:
    if request['amount'] > request['opening_balance']:
        print(f'Insufficient Funds : Cannot Withdraw {request['amount']}')
        request['new_balance'] = request['opening_balance']
    else:
        new_bal = request['opening_balance']-request['amount']
        print(f"Depositing {request['amount']}-> New balance:{new_bal}")
        request['new_balance'] = new_bal
    return request

In [6]:
graph = StateGraph(TransactionRequest)
graph.add_node('deposit', handle_deposit)
graph.add_node('withdraw', handle_withdraw)


In [7]:
graph.add_conditional_edges(START, categorize_transaction)
graph.add_edge('deposit', END)
graph.add_edge('withdraw', END)

In [8]:
runnable = graph.compile()

In [9]:
print("\n----Deposit-----")
print(runnable.invoke({
    "action":"deposit",
    "opening_balance":5000,
    "amount":1200,
    'new_balance':0
}))


----Deposit-----
Transaction Request:{'action': 'deposit', 'opening_balance': 5000, 'amount': 1200, 'new_balance': 0}
Depositing 1200-> New balance:6200
{'action': 'deposit', 'opening_balance': 5000, 'amount': 1200, 'new_balance': 6200}


In [10]:
print("\n----Withdraw-----")
print(runnable.invoke({
    "action":"withdraw",
    "opening_balance":4000,
    "amount":1200,
    'new_balance':0
}))


----Withdraw-----
Transaction Request:{'action': 'withdraw', 'opening_balance': 4000, 'amount': 1200, 'new_balance': 0}
Depositing 1200-> New balance:2800
{'action': 'withdraw', 'opening_balance': 4000, 'amount': 1200, 'new_balance': 2800}


In [11]:
from langchain_core.runnables.graph import MermaidDrawMethod
from PIL import Image
from io import BytesIO

In [12]:
graph_image = runnable.get_graph().draw_mermaid_png(
    draw_method=MermaidDrawMethod.API,
    output_file_path='graph.png'
)

In [13]:
img = Image.open(BytesIO(graph_image))
img.show()

In [ ]:
graph_image = Image.open(BytesIO(graph_image))
